<a href="https://colab.research.google.com/github/javiermv/myci/blob/main/Copia%20de%20Programa_Mami_OK.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title PASO 1:Inicialización del notebook
#@markdown > Las **INSTRUCCIONES** para generar una nueva campaña están al final de este notebook.

#@markdown > Esta celda se puede ejecutar sin riesgo de que sobreescriba valores viejos
id_solicitudes_de_inscripcion_2024C2 = '1byKUn6Cg91LlN7LomeeFfXnqQ5F5W3avWvVjM8GE4DU'
id_solicitudes_de_inscripcion = id_solicitudes_de_inscripcion_2024C2
#id_requerimientos_de_inscripcion_generado = '1IxZ8frxC5gC41Npr3s67vrgmQmSwEJEicEoMYKS1nP0'
#id_requerimientos_de_inscripcion_manual = '1-gE4Nemi4qECqQBjFHQml4lEyF6k03QNBqqTS4Ojcuc'
id_requerimientos_de_inscripcion_2024C2 = '1Uwkhl8bilkuL1apHZRn7mevZ5CCE_SvmQZhQBuFmLAI'
id_requerimientos_de_inscripcion = id_requerimientos_de_inscripcion_2024C2
#id_2023C1_ORDENES_DE_INSCRIPCION_CON_ERRORE = '1uYK73OpeGNyeIhR2MU5hJeQw7C_WK0fOUMUhdyfB-p4'
id_ordenes_de_inscripcion_2024C2= '1xqXdDRKgteq_EzdGaNFpm7s2cls-xFgw86vTwIKRvAQ' # acá borré la dirección porque no quería sobreescribir sobre el archivo de Laura. VER CON JMV
id_ordenes_de_inscripcion = id_ordenes_de_inscripcion_2024C2
id_2024C1_incoming_confirmados = ''# acá borré la dirección porque no quería sobreescribir archivo viejo
id_problemas_moodle = ''# acá borré la dirección porque no quería sobreescribir archivo viejo
####################################################
# si no vamos a usar el google-drive para desarrollar, quizás no sea necesario este 'mount'.
#from google.colab import drive
#drive.mount('/content/drive')
# después, revisar si hace falta este 'auth', habiendo deshabilitado el 'mount'.
#from google.colab import auth
#auth.authenticate_user()
############################################################
# Se definen estas funciones para que GoogleColab las conozca.
# Se redefinen en 'funciones.py' de github.
def leer_google_spreadsheet(doc_id : str, sheet_name : str = None, forget_previous : bool = False) -> list : return []
def leer_requerimientos() -> dict : return {}
def leer_ordenes_SIU3() -> dict :  return {}
def leer_encabezado_incoming() -> list : return []
def leer_listado_incoming() -> dict : return {}
def find_changes(new_row, old_row) -> dict : return {}
def explain_changes(header: list, changes: dict) -> list : return []
def guardar_dict_en_google_spreadsheet(sheets : dict, doc_id : str) : return
def guardar_list_en_google_spreadsheet(rows : list, doc_id : str, sheet_name = None) : return
############################################################

escuelas = [
        "EH - Escuela de Humanidades",
        "EPyG - Escuela de Política y Gobierno",
        "EAyP - Escuela de Arte y Patrimonio",
        "EIDAES - Escuela Interdisciplinaria de Altos Estudios Sociales",
        "ECyT - Escuela de Ciencia y Tecnología",
        "EEyN - Escuela de Economía y Negocios",
        "ICRM",
        "EHyS - Escuela de Hábitat y Sustentabilidad",
    ]

import urllib.request
def load_script_from_url(url):
    response = urllib.request.urlopen(url)
    script_code = response.read().decode('utf-8')
    # print(script_code)
    exec(script_code, globals())

!pip install xmltodict
load_script_from_url('https://raw.githubusercontent.com/javiermv/myci/main/funciones.py')

#@title Lectura de `incoming/listado.php`.
#@markdown > Genera la variable `incoming` como diccionario. La clave es el pasaporte.
#@markdown > Entre otras cuestiones falta definir qué hacer con las personas sin pasaporte.

_incoming_recordado = None

def leer_listado_incoming_nuevo(reportar_sin_pasaporte: bool = False) -> dict :
    #!pip install xmltodict

    global _incoming_recordado

    if _incoming_recordado != None :
        print('Se recordó el contenido de Incoming.')
        return _incoming_recordado

    #from xml.dom import minidom
    import requests
    import xmltodict

    # reportar_sin_pasaporte = False

    # en listado.php falta:
    #   - <! DOCTYPE html>
    #   - La codificación de caracteres del documento no se declaraba
    url = "http://www.unsam.edu.ar/internacional/incoming/listado.php"
    response = requests.get(url) # TARDA
    encoding = response.apparent_encoding
    if (encoding != 'utf-8') :
        print('Encoding aparente extraño: ', encoding)
        print('Se esperaba el encoding utf-8.')
    # encoding aparente anterior: Windows-1252
    # encoding aparente actual: utf-8

    # FALLAN MISERABLEMENTE
    #dom = minidom.parseString(str)
    #data = xmltodict.parse(response.content, encoding='Windows-1252')

    #str = response.content.decode('Windows-1252')
    str = response.content.decode(encoding)
    str = str.replace("border=1", "border='1'") # necesita corregirse el entrecomillado de los atributos.
    str = str.replace("&", "&#38;") # necesita corregirse la codificación del '&'.
    str = str.replace('<meta charset="UTF-8">', '<meta charset="UTF-8" />')
    str = str.replace('<meta name="viewport" content="width=device-width, initial-scale=1.0">', '')
    #print(str)
    # dom = minidom.parseString(str)
    # print(dom)
    # #dom.getElementsByTagName("tr")[0]
    # print(dom.getElementsByTagName("tr")) # tr: table-row
    # for elem in dom.getElementsByTagName("tr") : # tr: table-row # FUNCIONA
    #     elem.getElementsByTagName...

    dict = xmltodict.parse(str)
    #print(dict)
    # antes {'table': {'@border': '1', 'tr': [{'td': ['Fecha', 'Apellido', 'Nombre', 'Nacionalidad', 'Carrera', 'Universidad', 'Pais', 'Email', 'Email Alternativo', 'Periodo', 'Anio', 'Sexo', 'Pasaporte', 'Fecha Nacimiento', 'E-Mail Responsable']}, {'td': [None, 'López', 'Cecilia', 'Argentina', 'Psicopedagogía', 'Free Mover', 'Argentina', 'snielson@unsam.edu.ar', 'snielson@unsam.edu.ar', None, None, None, None, None, None]}, , ...}]}}
    # ahora {'html': {'@lang': 'en', 'head': {'meta': {'@charset': 'UTF-8'}, 'title': 'Listado'},
    #   'body': {'table': {'@border': '1', 'tr': [
    #       {'td': ['Fecha', 'Apellido', 'Nombre', 'Nacionalidad', 'Carrera', 'Universidad', 'Pais', 'Email', 'Email Alternativo', 'Periodo', 'Anio', 'Sexo', 'Pasaporte', 'Fecha Nacimiento', 'E-Mail Responsable']},
    #       {'td': [None, 'López', 'Cecilia', 'Argentina', 'Psicopedagogía', 'Free Mover', 'Argentina', 'snielson@unsam.edu.ar', 'snielson@unsam.edu.ar', None, None, None, None, None, None]},
    #       {'td': [...]}, ... ]}}}}
    dict = dict['html']['body']
    #print(dict)
    dict = dict['table']['tr'] # lista de {'td': [valores]}
    #print(dict)
    #  {'td': [None, 'López', 'Cecilia', 'Argentina', 'Psicopedagogía', 'Free Mover', 'Argentina', 'snielson@unsam.edu.ar', 'snielson@unsam.edu.ar', None, None, None, None, None, None]}, {'td': [None, 'Rodríguez Sánchez', 'Adrián Gerardo', 'Mexicana', 'Posgrado en Historiografía', 'Free Mover', 'Argentina', 'adrian.geros@gmail.com', 'adrian.geros@gmail.com', None, None, None, None, None, None]}, ...]}]
    encabezado = dict.pop(0)['td'] # ['Fecha', 'Apellido', 'Nombre', 'Nacionalidad', 'Carrera', 'Universidad', 'Pais', 'Email', 'Email Alternativo', 'Periodo', 'Anio', 'Sexo', 'Pasaporte', 'Fecha Nacimiento', 'E-Mail Responsable']
    index_pasaporte = encabezado.index('Pasaporte')

    # Diccionario Pasaporte -> Datos personales
    incoming = { }

    for row in dict :
        #values = row['td']     #[None, 'López', 'Cecilia', 'Argentina', 'Psicopedagogía', 'Free Mover', 'Argentina', 'snielson@unsam.edu.ar', 'snielson@unsam.edu.ar', None, None, None, None, None, None]
        #     pasaporte = tr['td'][index_pasaporte]

        persona = { }
        for index, value in enumerate(row['td']) :
            if value != None :
                persona[encabezado[index]] = value
            # si no hay pasaporte, ¿usamos 'Email'?
            if index == index_pasaporte and value == None and reportar_sin_pasaporte:
                print("Esta persona no tiene definido el pasaporte: ", persona)
        if 'Pasaporte' in persona :
            incoming[persona['Pasaporte']] = persona

    print('Leído:', url)
    #print("La variable 'incoming' es un diccionario Pasaporte -> DatosPersonales. ")
    print("El resultado de leer Incoming es un diccionario Pasaporte -> DatosPersonales. ")
    print('Las claves del diccionario DatosPersonales son: ', encabezado)
    print('Por ejemplo: "incoming[1234] == { Apellido : Messi, Nombre : Lionel, Pais : Argentina }".')

    _incoming_recordado = incoming
    return incoming

def locate_persona(pasaporte_req, requerimientos, incoming) -> dict :
    # podrían definirse 'requerimientos' e 'incoming' como optativos, e ir a
    # buscar sus contenidos si no se reciben.
    # print('pasaporte_req', pasaporte_req)
    # print('incoming', incoming)
    if pasaporte_req in incoming :
        return incoming[pasaporte_req]

    for pasaporte in incoming.keys() :
        if pasaporte.startswith(pasaporte_req) or pasaporte_req.startswith(pasaporte) :
            print('El pasaporte en Requerimientos', pasaporte_req,
                  'no coincide con el pasarte en Incoming', pasaporte,
                  'pero uno es un prefijo del otro. Se usan datos de Incoming.')
            return incoming[pasaporte]

    #requerimientos = leer_requerimientos()
    req = requerimientos[pasaporte_req]
    correo_req = req['Correo']
    # print('correo_req', correo_req)
    # encabezado = requerimientos[0]
    # index_pasaporte = encabezado.index('Pasaporte')
    # req = next(row for row in requerimientos
    #            if row[index_pasaporte] == pasaporte_req)
    # index_correo = encabezado.index('Correo')
    # correo_req = req[index_correo]

    # matches = [persona for persona in incoming.values()
    #         if persona['Email'] == correo_req
    #     ]
    matches = [(pasaporte, persona) for (pasaporte, persona) in incoming.items()
            if persona['Email'] == correo_req
        ]
    if len(matches) == 0 :
        matches = [(pasaporte, persona) for (pasaporte, persona) in incoming.items()
            if 'Email Alternativo' in persona and
                persona['Email Alternativo'] == correo_req
        ]
        # matches = [persona for persona in incoming.values()
        #     if 'Email Alternativo' in persona and
        #         persona['Email Alternativo'] == correo_req
        # ]
    if len(matches) == 1 :
        #print('matches[0]', matches[0])
        # matches[0] == ( pasaporte, incoming[pasaporte] )
        # pasaporte_incoming = matches[0]['Pasaporte']
        pasaporte_incoming = matches[0][0]
        print('El pasaporte en Requerimientos', pasaporte_req, 'no coincide con',
              'el pasaporte en Incoming', pasaporte_incoming, 'pero tienen el'
              'mismo correo', correo_req, '. Se usan datos de Incoming.')
        # Varias veces se encuentran en Incoming pasaportes que son prefijos o
        # sufijos de lo cargado en Requerimientos. Xej:
        # Pasaporte en Requerimientos: N04886292 , Pasaporte en Incoming: 4886292
        # También hay problemas de mayúsculas y minúsculas. Xej:
        # Pasaporte en Requerimientos: c77w8pkvx , Pasaporte en Incoming: C77W8PKVX
        return incoming[pasaporte_incoming]
    if (len(matches) > 1) :
        print ('Situación inesperada. No sé qué hacer. Se encontraron más de',
               'una coincidencia en Incoming para el correo', correo_req)
        print('matches:', matches)
    else :
        if (not pasaporte_req in pasaportes_faltantes) :
            print('No existe estudiante en Incoming con el pasaporte informado en',
                'Requerimientos', pasaporte_req, 'ni con el correo', correo_req,
                '. Datos completos del estudiante: ', req)
            pasaportes_faltantes.append(pasaporte_req)
    return None

# En el paso 4 (Generación de Resúmenes de Inscripción) se llama a generar_resumenes
# basicos sin parámetros, después de haber leído requerimientos e incoming. NO
# se entiende qué valores usa ni por qué acá adentro se vuelve a leer requerimientos.
# Probamos pasarle los reqs como parámetro y se rompió xq esta función usa el
# encabezado de leer_google_spreadsheet. Revisar.
#def generar_resumenes_basicos(requerimientos) -> dict :
def generar_resumenes_basicos() -> dict :

    # 'resumen' es un Diccionario (Escuela -> Resumen). Además, se define una
    # _escuela especial_ ("UNSAM") con un Resumen levemente distinto. Esta
    # _escuela especial_ guarda la información combinada de todas las escuelas.
    # Resumen es, a su vez, otro diccionario.
    resumen = { }
    for escuela in escuelas :
        # Esta información sirve para cruzar información con los datos de incoming.
        # Xej, con los pasaportes se obtienen las nacionalidades.
        resumen[escuela] = {
            'cantidad_ordenes' : 0,
            'pasaportes' : set(),   # es un conjunto
            'materias' : { }        # dict: Materia -> Conjunto de pasaportes
        }
    resumen['UNSAM'] = {
        # No es claro que sea útil pero las claves se definen en mayúsculas para
        # no confundirlas con las de los resúmenes de las escuelas.
            'CANTIDAD_ORDENES' : 0,
            'PASAPORTES' : set ()
        }

    requerimientos = leer_google_spreadsheet(id_requerimientos_de_inscripcion)
    encabezado = requerimientos[0]
    #print(encabezado) # ['Correo', 'Nombre', 'Pasaporte', 'Escuela', 'Codigo', 'Materia', 'Horario', 'Prioridad']
    index_escuela   = encabezado.index('Escuela') # es el nro de columna que en el encabezado se llama 'Escuela'
    index_pasaporte = encabezado.index('Pasaporte')
    index_materia   = encabezado.index('Materia')

    for row in requerimientos[1:] : # saltea el encabezado (requerimientos[0])
        escuela = row[index_escuela]
        if escuela == '' :
            print('row raro xq la escuela está vacía: ', row)
            break
        if not escuela in resumen :
            print('row raro xq la escuela no se conoce: ', row)
            print('escuela detectada: ', escuela)
            print('escuelas conociddas: ', escuelas)

        resumen[escuela]['cantidad_ordenes'] = resumen[escuela]['cantidad_ordenes'] + 1
        resumen['UNSAM']['CANTIDAD_ORDENES'] = resumen['UNSAM']['CANTIDAD_ORDENES'] + 1

        pasaporte = row[index_pasaporte]
        #print(pasaporte, escuela)
        resumen[escuela]['pasaportes'].add(pasaporte)
        resumen['UNSAM']['PASAPORTES'].add(pasaporte)
        materia = row[index_materia]
        # materias_escuela es un diccionario (Materia -> Conjunto de pasaportes)
        # Que sea un Conjunto permite evitar repeticiones.
        materias_escuela = resumen[escuela]['materias']
        if not materia in materias_escuela :
            materias_escuela[materia] = set ()
        materias_escuela[materia].add(pasaporte)

    print('Resúmenes Básicos generados.')
    return resumen


In [ ]:
#@title PASO 2: Genera los Requerimientos de inscripción a partir de las Solicitudes

# #------------------
rows = leer_google_spreadsheet(id_solicitudes_de_inscripcion)
datos_materia=['Escuela', 'Codigo', 'Materia', 'Horario', 'Prioridad']
#datos_materia=['Escuela', 'Codigo', 'Materia', 'Horario', 'Prioridad','Decision']
encabezado = ['Correo', 'Nombre', 'Pasaporte'] + datos_materia +['Problemas']
#encabezado = ['Correo', 'Nombre', 'Pasaporte', 'Escuela', 'Codigo', 'Materia', 'Horario', 'Prioridad','Decision', 'Problemas']
out_rows = [encabezado] # es una lista de listas
# Aca voy a poner el valor del i que es donde comienza la primera materia
primera_materia = 4
cantidad_materias = 6
columnas_x_materias = len(datos_materia)
#info_materia tiene los indices donde empieza la información de las distintas mateiras
info_materia = []
for j in range (cantidad_materias) :
    info_materia = info_materia + [primera_materia + j * columnas_x_materias]
for row in rows[1:]: # saltea encabezado; cada fila es una lista de Strings
    #print('row: ', row)
    if len(row) < primera_materia + columnas_x_materias * cantidad_materias:
        print(f'Fila con datos insuficientes: {row}')
        continue  # Saltear filas que no tienen suficientes columnas
    correo = row[1]
    nombre = row[2]
    pasaporte = row[3]
    for i in info_materia:
        escuela = row[i]
        código = row[i+1]
        materia = row[i+2]
        horario = row[i+3]
        prioridad = row[i+4]
        #decision = row[i+5]
        requerimiento = [correo, nombre, pasaporte, escuela, código, materia, horario, prioridad]
        #requerimiento = [correo, nombre, pasaporte, escuela, código, materia, horario, prioridad, decision]
        #print(','.join(requerimiento))
        out_rows.append(requerimiento)

prevs = leer_google_spreadsheet(id_requerimientos_de_inscripcion, forget_previous = True)

# comienza la combinación de los resultados nuevos con los previos (si existen)
if len(out_rows) < len(prevs) :
    raise ValueError("Se asume que los nuevos Requerimientos no pueden ser menos que los anteriores.")

skipped = 1 # abajo se saltea el encabezado; debería considerarse especialmente.
for i in range(1, min(len(out_rows), len(prevs))) :
    changes = find_changes(out_rows[i], prevs[i])
    if not changes :
        #print('se saltea la línea', i)
        skipped += 1
        continue
    #print('out', i, out_rows[i])
    #print('pre', i, prevs[i])
    explicacion = 'Cambios en: ' + (', '.join(explain_changes(encabezado, changes)))
    idx = encabezado.index('Problemas')
    print('Hay diferencias en la fila', i + 1, ':', explicacion)
    prev_problema = prevs[i][idx]
    if prev_problema != "" :
        prev_problema += ". "
    out_problema = prev_problema + explicacion
    out_rows[i].append(out_problema)
    #print('OUT', i, out_rows[i])

if skipped == len(out_rows) :
    print('No se modificaron los Requerimientos ya que todas las filas coincidían con las Solicitudes.')
else :
    # imprimir cuantas de las filas modificadas en realidad son nuevas.
    guardar_list_en_google_spreadsheet(out_rows, id_requerimientos_de_inscripcion)
    print('Se modificaron', len(out_rows)-skipped, 'filas.')
    print('Se generaron los Requerimientos de inscripción.')

# imprimir url clicleable de la hoja generada.
# imprimir fecha actual cada vez que se corre.
# detectar formularios completados varias veces x la misma persona.


Se leyó la hoja Respuestas de formulario 1 de 2024-C2-Solicitud de inscripción (respuestas)
Se leyó la hoja Hoja 1 de 2024-C2-Requerimientos de inscripcion
Hay diferencias en la fila 221 : Cambios en: Codigo: MA-TE00036 -> PEDC015, Materia: ecnologías emergentes (Licenciatura en Prácticas Artísticas Contemporáneas) -> Dinámicas del cuerpo (Licenciatura en Prácticas Artísticas Contemporáneas)
Hay diferencias en la fila 222 : Cambios en: Codigo: MA-AH00000 -> MA-I000017, Materia: Arte y humor (Licenciatura en Artes Escénicas -> Instalación (Licenciatura en Arte Contemporáneo), Horario: Jueves: 14 - 18 hs -> Lunes: 9 - 13 hs
Hay diferencias en la fila 262 : Cambios en: Codigo: MA-TE00036 -> MA28, Materia: Tecnologías emergentes (Licenciatura en Prácticas Artísticas Contemporáneas) -> Composición Comisión 1, Horario: Miércoles 9 a 13 -> Jueves 9 a 11
Se escribieron 295 filas en la hoja Hoja 1 de 2024-C2-Requerimientos de inscripcion
Se modificaron 3 filas.
Se generaron los Requerimientos d

In [ ]:
#@title PASO3: Generación de Ordenes de inscripción SIU2 y SIU3 a partir de los Requerimientos de Inscripción
###############################################
# Esta versión es más nueva que la de github/larawb/ProgramaMami.ipynb

#encabezado_SIU2 = ['Nombre', 'Pasaporte', 'Materia', 'Horario', 'Prioridad']
encabezado_SIU2 = ['Nombre', 'Pasaporte', 'Materia', 'Horario', 'Prioridad', 'Decisión','Actualizaciones x sistema']
encabezado_SIU3 = ['Correo','Nombre', 'Pasaporte', 'Escuela', 'Codigo', 'Materia', 'Horario', 'Prioridad', 'Decisión', 'Actualizaciones x sistema']
#encabezado_SIU3 = ['Correo','Nombre', 'Pasaporte', 'Escuela', 'Codigo', 'Materia', 'Horario', 'Prioridad', 'Estado', 'Observaciones']
# anterior #encabezado_SIU3 = ['Fecha_nacimiento','Universidad', 'Correo', 'Nombre', 'Pasaporte', 'Escuela', 'Codigo', 'Materia', 'Horario', 'Prioridad', 'Estado', 'Observaciones']

# codigo_escuela relaciona el nombre de la Escuela con la pestaña en el spreadsheet
codigo_escuelas_SIU2 = {
        'EIDAES - Escuela Interdisciplinaria de Altos Estudios Sociales' : 'EIDAES',
        'EPyG - Escuela de Política y Gobierno' : 'EPYG',
        #'ECyT - Escuela de Ciencia y Tecnología' : 'ECYT',
        #'EHyS - Escuela de Hábitat y Sustentabilidad' : 'EHYS'
    }

resultados_SIU2 = {}
# genera los encabezados de todas las escuelas siu2
for nombre, codigo in codigo_escuelas_SIU2.items() :
    resultados_SIU2[codigo] = [encabezado_SIU2]
# genera el encabezado unico para todas las escuelas siu3
out_rows_SIU3 = [encabezado_SIU3]

#incoming = leer_listado_incoming()   # FIXME: no está funcionando
#requerimientos = leer_requerimientos() # FIXME: no está funcionando

# Por ahora se sigue usando leer_google_spreadsheet() para limpiar ESTA función
# sin cambiar su comportamiento; quizás después se use solo leer_requerimientos()
rows = leer_google_spreadsheet(id_requerimientos_de_inscripcion)

for row in rows[1:]: # saltea encabezado; cada fila es una lista de Strings
    #Correo(0)	Nombre (1)	Pasaporte(2)	Escuela(3)	Codigo(4)	Materia(5)	Horario (6)	Prioridad(7) Decisión(8)
    #out_humanidades.append(resultado) # este se vanombre = row[2]
    nombre = row[1]
    pasaporte = row[2]
#    persona = locate_persona(pasaporte, requerimientos, incoming) # FIXME: no está funcionando
#    if pasaporte != persona['Pasaporte'] :
#        print('Reemplazando el pasaporte', pasaporte, 'por el registrado en Incoming:', persona['Pasaporte'])
#        pasaporte = persona['Pasaporte']
# Para completar la universidad y fecha de nacimiento necesito que funcione la lectura de incoming
#    universidad = persona['Universidad'] if 'Universidad' in persona else ''
    #universidad = '' # esta es la linea ok si tengo que volver a usar fecha de nacimiento #FIXME: no está funcionando Incoming
    # if pasaporte in universidades :
    #     universidad = universidades[pasaporte]
    # else :
    #     universidad = ''
#    fecha_nacimiento = persona['Fecha Nacimiento'] if 'Fecha Nacimiento' in persona else '' # FIXME
    #fecha_nacimiento = '' # esta es la linea ok si tengo que volver a usar fecha de nacimiento
        # if pasaporte in fechas_nacimiento :
    #     fecha_nacimiento = fechas_nacimiento[pasaporte]
    # else :
    #     fecha_nacimiento = ''

    escuela = row[3]
    materia = row[5]
    horario = row[6]
    prioridad = row[7]
    #decision = row[8]

    if escuela in codigo_escuelas_SIU2 :
        #orden_SIU2 = [nombre, pasaporte, materia,horario, prioridad, decision]
        orden_SIU2 = [nombre, pasaporte, materia,horario, prioridad]
        resultados_SIU2[codigo_escuelas_SIU2[escuela]] += [ orden_SIU2 ]
    else :
        # Formato SIU3: Correo, Nombre,Pasaporte,Escuela,Codigo,Materia,Horario,Prioridad,Estado de la inscripción
        correo = row[0]
        codigo = row[4]
        #orden_SIU3 = [correo, nombre, pasaporte, escuela, codigo, materia, horario, prioridad,decision]
        orden_SIU3 = [correo, nombre, pasaporte, escuela, codigo, materia, horario, prioridad]
        ## orden_SIU3 = [fecha_nacimiento, universidad, correo, nombre, pasaporte, escuela, codigo, materia, horario, prioridad]
        out_rows_SIU3.append(orden_SIU3)

# acá comienza la combinación entre los valores nuevos y los viejos (si existen) para SIU3
#dict_ordenes_SIU3 = leer_ordenes_SIU3() # dict: pasaporte -> [Eleccion]
prevs = leer_google_spreadsheet(id_ordenes_de_inscripcion, 'SIU3', False)
encabezado_ordenes_SIU3 = out_rows_SIU3[0]
skipped = 1 # abajo se saltea el encabezado; debería considerarse especialmente.
for i in range(1, min(len(out_rows_SIU3), len(prevs))) :
    # print('i:', i, ', out_row_SIU3[i]:', out_rows_SIU3[i])
    # print('i:', i, ', prevs[i]:', prevs[i])
    changes = find_changes(out_rows_SIU3[i], prevs[i])
    if not changes :
        skipped += 1
        continue

    explicacion = 'Cambios en: ' + (', '.join(explain_changes(encabezado_ordenes_SIU3, changes)))
    idx_actualizacion = encabezado_ordenes_SIU3.index('Actualizaciones x sistema')
    print('Hay diferencias en la fila', i+1, ':', explicacion)
    #print('actual', i+1, out_rows_SIU3[i])
    print('previo', i+1, prevs[i])
    prev_actualizacion = prevs[i][idx_actualizacion]
    if prev_actualizacion != "" :
        prev_actualizacion += ". "
    out_actualizacion = prev_actualizacion + explicacion
    # out_rows[i].append(out_problema)

    if len(out_rows_SIU3[i]) <= idx_actualizacion:
        out_rows_SIU3[i].extend([''] * (idx_actualizacion - len(out_rows_SIU3[i]) + 1))
    out_rows_SIU3[i][idx_actualizacion] = out_actualizacion
    #idx_decision = encabezado_ordenes_SIU3.index('Decisión')
    #out_rows_SIU3[i][idx_decision] = prevs[i][idx_decision]

    #print('encabezado:', encabezado_ordenes_SIU3)
    print('nuevo ', i+1, out_rows_SIU3[i])

print("----------------------")
resultados = resultados_SIU2

if skipped == len(out_rows_SIU3) :
    print('No se modificaron las Ordenes de SIU3 ya que todas las filas coincidían con los nuevos Requerimientos.')
else :
    resultados["SIU3"] = out_rows_SIU3
################################################
# Acá se guardan los resultados en las planillas.
guardar_dict_en_google_spreadsheet(resultados, id_ordenes_de_inscripcion)
# # acá empezamos a probar si podemos agregar, con este programa, una lista de valores de validación:
# #   'Inscripto, Pendiente, Problemas, Baja'
# #get_data_validation_rule(hoja de trabajo, etiqueta)
# #set_data_validation_for_cell_range(hoja de trabajo, rango, regla)
# #set_data_validation_for_cell_ranges(hoja de trabajo, rangos)
# #rule = get_data_validation_rule(sheet, 'I2')
# cell = sheet.cell(2, 9)
# #print(cell) # <Cell R2C9 'Pendiente'>
# #print(sheet.acell('I2')) # <Cell R2C9 'Pendiente'>
# sheet.format("I2", {"textFormat": {"bold": True}})
# sheet.get("I2")
# sheet.format("I2", {"textFormat": {"bold": True}}) # quizás sigamos x acá
print('te quiero')

# imprimir fecha actual cada vez que se corre


Se recordó el contenido de la hoja Hoja 1 de 2024-C2-Requerimientos de inscripcion
Se leyó la hoja SIU3 de 2024-C2-Ordenes de Inscripción
Hay diferencias en la fila 140 : Cambios en: Codigo: MA-TE00036 -> PEDC015, Materia: ecnologías emergentes (Licenciatura en Prácticas Artísticas Contemporáneas) -> Dinámicas del cuerpo (Licenciatura en Prácticas Artísticas Contemporáneas)
previo 140 ['maitek@web.de', 'Maite Küpper', 'C122Z8PTP', 'EAyP - Escuela de Arte y Patrimonio', 'MA-TE00036', 'ecnologías emergentes (Licenciatura en Prácticas Artísticas Contemporáneas)', 'Miércoles: 9-13 hs', 'Suplente', '', '', 'activado', 'inscripto', 'aceptada']
nuevo  140 ['maitek@web.de', 'Maite Küpper', 'C122Z8PTP', 'EAyP - Escuela de Arte y Patrimonio', 'PEDC015', 'Dinámicas del cuerpo (Licenciatura en Prácticas Artísticas Contemporáneas)', 'Miércoles: 9-13 hs', 'Suplente', '', 'Cambios en: Codigo: MA-TE00036 -> PEDC015, Materia: ecnologías emergentes (Licenciatura en Prácticas Artísticas Contemporáneas) -

In [ ]:
#@title PASO 4: Generación de Resúmenes de Inscripción
##############################################

# Si la próxima línea falla con:
#   NameError: name 'incoming' is not defined
# es xq falta ejecutar la celda 'Lectura de incoming/listado.php'.
#print(incoming)

#incoming = leer_listado_incoming()         # en github, hay que actualizarlo.
incoming = leer_listado_incoming_nuevo()    # definido en el Paso 1, falta fusionarlo con la versión en github
#print(incoming)

requerimientos = leer_requerimientos()
# print('incoming', incoming)
# Lista de personas no identificadas in incoming/listado.php
pasaportes_faltantes =  [ ]

sinonimo_nacionalidad = {
    # para agregar sinónimos la fórmula es 'si decía X' : 'poner Y'
    'alemania' : 'alemana','german':'alemana','germany':'alemana',
    'brazilian':'brasilera',
    'france' : 'francesa', 'french' : 'francesa', 'francia' : 'francesa','française':'francesa',
    'españa' : 'española',
    'mexicano' : 'mexicana',
    'polish':'polaca',
    'united kingdom':'britanica'
}

#resumenes = generar_resumenes_basicos(requerimientos) # sería lindo usar esto
resumenes = generar_resumenes_basicos()

#for escuela in resumenes.keys() :
for escuela in escuelas :
    # Si la próxima línea falla con:
    #   NameError: name 'resumenes' is not defined
    # es xq falta ejecutar la celda 'Generación de Resúmenes Básicos de Inscripción'.
    resumen = resumenes[escuela]
    resumen['generos'] = { }
    generos = resumen['generos']
    nacionalidades = resumen['nacionalidades'] = { }
    for pasaporte in resumen['pasaportes'] :
        persona = locate_persona(pasaporte.strip(), requerimientos, incoming)
        if persona == None:
            continue
        # print('Persona:', persona)
        # genero = 'Otro'
        genero = 'Desconocido'
        if 'Sexo' in persona :
            genero = persona['Sexo']
        if not genero in generos :
            generos[genero] = 0
        generos[genero] += 1

        nacionalidad = 'desconocida'
        if 'Nacionalidad' in persona :
            nacionalidad = persona['Nacionalidad'].lower()
            if nacionalidad in sinonimo_nacionalidad :
                nacionalidad = sinonimo_nacionalidad[nacionalidad]
        if not nacionalidad in nacionalidades :
            nacionalidades[nacionalidad] = 0
        nacionalidades[nacionalidad] += 1


# Si la próxima línea falla con:
#   NameError: name 'resumen_UNSAM' is not defined
# es xq falta ejecutar la celda 'Generación de Resúmenes Básicos de Inscripción'.
# GENEROS_UNSAM = resumen_UNSAM['GENEROS'] = { }
# NACIONALIDADES_UNSAM = resumen_UNSAM['NACIONALIDADES'] = { }
# UNIVERSIDADES_UNSAM = resumen_UNSAM['UNIVERSIDADES'] = { }
GENEROS_UNSAM = resumenes['UNSAM']['GENEROS'] = { }
NACIONALIDADES_UNSAM = resumenes['UNSAM']['NACIONALIDADES'] = { }
UNIVERSIDADES_UNSAM = resumenes['UNSAM']['UNIVERSIDADES'] = { }

# resumen_UNSAM = {
#         'CANTIDAD_ORDENES' : 0,
#         'PASAPORTES' : set ()
#     }
for pasaporte in resumenes['UNSAM']['PASAPORTES'] :
    persona = locate_persona(pasaporte.strip(), requerimientos, incoming)
    if persona == None:
        continue
    # print('Persona:', persona)
    genero = 'Desconocido'
    if 'Sexo' in persona :
        genero = persona['Sexo']
    if not genero in GENEROS_UNSAM :
        GENEROS_UNSAM[genero] = 0
    GENEROS_UNSAM[genero] += 1

    nacionalidad = 'desconocida'
    if 'Nacionalidad' in persona :
        nacionalidad = persona['Nacionalidad'].lower()
    if nacionalidad in sinonimo_nacionalidad :
        nacionalidad = sinonimo_nacionalidad[nacionalidad]
    if not nacionalidad in NACIONALIDADES_UNSAM :
        NACIONALIDADES_UNSAM[nacionalidad] = 0
    NACIONALIDADES_UNSAM[nacionalidad] += 1

    universidad = 'Desconocida'
    if 'Universidad' in persona :
        universidad = persona['Universidad']
    if not universidad in UNIVERSIDADES_UNSAM :
        UNIVERSIDADES_UNSAM[universidad] = 0
    UNIVERSIDADES_UNSAM[universidad] += 1


print("No se encontraron en 'incoming' los pasaportes:", pasaportes_faltantes)
print("Las personas con pasaporte faltante o datos en blanco se asociaron con 'Desconocido/a' género, nacionalidad y universidad.")

print('UNSAM')
print('    #inscripciones:', resumenes['UNSAM']['CANTIDAD_ORDENES'])
generos = resumenes['UNSAM']['GENEROS']
generos['Total'] = len(resumenes['UNSAM']['PASAPORTES'])
print('    estudiantes:', generos)
print('    nacionalidades:', resumenes['UNSAM']['NACIONALIDADES'])
#print('    #materias:', len(resumen['materias']))
print('    Universidades:')
universidades = resumenes['UNSAM']['UNIVERSIDADES']
for u in universidades.keys() :
#    pasaportes_inscriptos = materias[m]
    print('     -', u + ':', universidades[u])


#for escuela in resumenes.keys() :
for escuela in escuelas :
    print(escuela)
    resumen = resumenes[escuela]
    print('    #inscripciones:', resumen['cantidad_ordenes'])
    generos = resumen['generos']
    generos['Total'] = len(resumen['pasaportes'])
    print('    estudiantes:', generos)
    print('    nacionalidades:', resumen['nacionalidades'])
    print('    #materias:', len(resumen['materias']))
    print('    Inscriptos:')
    materias = resumen['materias']
    for m in materias.keys() :
        pasaportes_inscriptos = materias[m]
        contactos = set()
        for pasaporte in pasaportes_inscriptos :
            # req = next(row for row in requerimientos if row[index_pasaporte] == pasaporte)
            # nombre = req[index_nombre]
            # correo = req[index_correo]
            req = requerimientos[pasaporte]
            nombre = req['Nombre']
            correo = req['Correo']
            contacto = nombre + ' <' + correo + '>'
            #contacto = 'persona(s) sin información'
            #if pasaporte in incoming :
            #    persona = incoming[pasaporte]
            #    #contacto = (persona['Nombre'], persona['Apellido'], persona['Email'])
            #    contacto = persona['Nombre'] + ' ' + persona['Apellido'] + ' <' + persona['Email'] + '>'
            contactos.add(contacto)
        contactos_text = ', '.join(contactos)
        print('      -', m, ':', contactos_text)



Leído: http://www.unsam.edu.ar/internacional/incoming/listado.php
El resultado de leer Incoming es un diccionario Pasaporte -> DatosPersonales. 
Las claves del diccionario DatosPersonales son:  ['Fecha', 'Apellido', 'Nombre', 'Nacionalidad', 'Carrera', 'Universidad', 'Pais', 'Email', 'Email Alternativo', 'Periodo', 'Anio', 'Sexo', 'Pasaporte', 'Fecha Nacimiento', 'E-Mail Responsable']
Por ejemplo: "incoming[1234] == { Apellido : Messi, Nombre : Lionel, Pais : Argentina }".
Se recordó el contenido de la hoja Hoja 1 de 2024-C2-Requerimientos de inscripcion
Ejemplo: requerimientos[21AD17569]: {'Correo': 'magali.ostuni@gmail.com', 'Nombre': 'Magali Ostuni ', 'Elecciones': [{'Escuela': 'EPyG - Escuela de Política y Gobierno', 'Codigo': '', 'Materia': 'POLÍTICA  LATINOAMERICANA', 'Horario': 'LUNES 8.00 a 12.00', 'Prioridad': 'Principal'}, {'Escuela': 'EPyG - Escuela de Política y Gobierno', 'Codigo': '', 'Materia': 'OPINIÓN PÚBLICA Y COMUNICACIÓN POLITICA', 'Horario': 'Martes 8.00 a 12.00', 

In [ ]:
#@title PASO 5: Generación de mensajes de devolución de inscripciones por estudiante
############################################################################
# Para generar estos mensajes es necesario leer las órdenes de cada escuela.
# En el caso del SIU3 tenemos una situación actualizada.

def convertir_en_dict(rows : list, key_name : str, cols : list) -> dict :
    result = { }
    encabezado = rows[0]
    for row in rows[1:] : # saltea encabezado
        value = { }
        #print('row', row)
        for col in cols :
            value[col] = row[encabezado.index(col)]

        key = row[encabezado.index(key_name)]
        # OJO que key o value pueden ser banana en filas con celdas vacías.

        if not key in result :
            result[key] = [ ]
        result[key].append(value)
    return result

def leer_ordenes_SIU2(sheet_name : str) -> dict :
    ordenes_SIU2 = leer_google_spreadsheet(id_ordenes_de_inscripcion, sheet_name)
# #   atributos_eleccion = ['Escuela', 'Codigo', 'Materia', 'Horario', 'Prioridad'] # SIU3
   # atributos_eleccion = ['Materia', 'Horario', 'Prioridad'] # SIU2
    atributos_eleccion = ['Materia', 'Horario', 'Decisión'] # SIU2
    result = convertir_en_dict(ordenes_SIU2, 'Pasaporte', atributos_eleccion)
    return result

escuelas_SIU2 = {
        'EIDAES' : 'Escuela Interdisciplinaria de Altos Estudios Sociales',
        'EPYG' : 'Escuela de Política y Gobierno',
        #'EHYS' : 'Escuela de Hábitat y Sustentabilidad'
    }

ordenes = leer_ordenes_SIU3() # dict : pasaporte -> [ orden ]

# se agregan las ordenes del siu2
for key in escuelas_SIU2.keys() :
    escuela = escuelas_SIU2[key]
    # print(escuela)
    ordenes_escuela = leer_ordenes_SIU2(key)
    for elecciones in ordenes_escuela.values() :
        for eleccion in elecciones :
            eleccion["Escuela"] = escuelas_SIU2[key]
    # print(ordenes_escuela)
    for pasaporte in ordenes_escuela.keys() :
        if not pasaporte in ordenes :
            ordenes[pasaporte] = [ ]
        ordenes[pasaporte] += ordenes_escuela[pasaporte]
        # print('pasaporte', pasaporte, 'ordenes', ordenes[pasaporte])

# No es necesario leer los Requerimientos pero resulta cómodo para acceder a la
# información personal asociada a cada pasaporte.
requerimientos = leer_requerimientos()
# requerimientos[pasaporte] = { 'Correo' : str, 'Nombre' : str, 'Elecciones' : [ Eleccion ] }
for pasaporte in ordenes.keys() :
    elecciones = ordenes[pasaporte]
    confirmadas = filter(lambda eleccion :
                            eleccion['Decisión'] == 'CONFIRMAR MATERIA', elecciones)
    #confirmadas = list(map(lambda eleccion : eleccion['Materia'], confirmadas))
    dadas_de_baja = list(filter(lambda eleccion :
                            eleccion['Decisión'] == 'DAR DE BAJA', elecciones))
    #dadas_de_baja = list(map(lambda eleccion : eleccion['Materia'], dadas_de_baja))
    otras = list(filter(lambda eleccion :
                            not eleccion['Decisión'] in ['CONFIRMAR MATERIA', 'DAR DE BAJA'],
                            elecciones))
    for otra in otras :
        print('La materia', otra['Materia'], 'no ha informado su decisión. Dice', otra['Decisión'],
              ', fuera de lo esperado: ["CONFIRMAR MATERIA", "DAR DE BAJA"].')

    # materias = list(map(lambda eleccion : eleccion['Materia'], ordenes[pasaporte]))
    # print('Pasaporte:', pasaporte, ', Materias:', materias)
    if pasaporte in requerimientos :
        nombre = requerimientos[pasaporte]['Nombre']
        correo = requerimientos[pasaporte]['Correo']

        #texto = correo + '\n' + nombre + '\n'
        texto = '11 de abril de 2024' + '\n'+ '\n'+ nombre + '\n'
        explicacion = '\n' "Estas son tus inscricipones definitivas. Es importante que completes en el formulario tu decisión sobre las materias que están listadas bajo el título 'Materias sin confirmar' para que podamos finalizar tu inscripción definitiva.  En caso de que encuentres algún error debes poner una 'X' al lado de la materia y describir cuál es el problema." + '\n'
        texto = texto + explicacion +'\n' + '\n'
        # no vamos a poner encabezado acá
        mis_elecciones = requerimientos[pasaporte]['Elecciones']
        """
        for mi_eleccion  in mis_elecciones:
            escuela = mi_eleccion['Escuela']
            materia = mi_eleccion['Materia']
            horario = mi_eleccion['Horario']
            texto = texto + ' [ ] ' + ', '.join([materia,horario,escuela])+'\n'+'\n'
        texto = texto + 'Firma' +'\n'+'\n'+'\n' +'Nombre y Apellido''\n'+'\n'+ "..."
        print(texto)
        """
        texto = texto + "Materias confirmadas: "
        elecciones = ""
        for mi_eleccion  in confirmadas:
            escuela = mi_eleccion['Escuela']
            materia = mi_eleccion['Materia']
            horario = mi_eleccion['Horario']
            elecciones = elecciones + ' [ ] ' + ', '.join([materia,horario,escuela])+'\n'

        if not elecciones :
            texto = texto + "NINGUNA" + '\n' + '\n'
        else :
            texto = texto + '\n' + elecciones + '\n' + '\n'

        texto = texto + "Materias dadas de baja: "
        elecciones = ""
        for mi_eleccion  in dadas_de_baja:
            escuela = mi_eleccion['Escuela']
            materia = mi_eleccion['Materia']
            horario = mi_eleccion['Horario']
            elecciones = elecciones + ' [ ] ' + ', '.join([materia,horario,escuela])+'\n'

        if not elecciones :
            texto = texto + "NINGUNA" + '\n' + '\n'
        else :
            texto = texto + '\n' + elecciones + '\n' + '\n'

        texto = texto + "Materias sin confirmar: "
        elecciones = ""
        for mi_eleccion  in otras:
            escuela = mi_eleccion['Escuela']
            materia = mi_eleccion['Materia']
            horario = mi_eleccion['Horario']
            elecciones = elecciones + ' [ ] ' + ', '.join([materia,horario,escuela])+'\n'

        if not elecciones :
            texto = texto + "NINGUNA" + '\n' + '\n'
        else :
            texto = texto + '\n' + elecciones + '\n' + '\n'

        texto = texto + 'Firma' +'\n'+'\n'+'\n' +'Nombre y Apellido''\n'+'\n'+ "..."
        print(texto)

    else :
        print('El pasaporte no se encuentra en los Requerimientos:', pasaporte)




Se leyó la hoja SIU3 de 2024-C2-Ordenes de Inscripción


ValueError: 'Estado' is not in list

# Nueva sección

In [ ]:
#@title PASO QUIQUE: Generación de Resúmenes de Inscripción
##############################################

# Si la próxima línea falla con:
#   NameError: name 'incoming' is not defined
# es xq falta ejecutar la celda 'Lectura de incoming/listado.php'.
#print(incoming)

#incoming = leer_listado_incoming()         # en github, hay que actualizarlo.
incoming = leer_listado_incoming_nuevo()    # definido en el Paso 1, falta fusionarlo con la versión en github
#print(incoming)

requerimientos = leer_requerimientos()
# print('incoming', incoming)
# Lista de personas no identificadas in incoming/listado.php
pasaportes_faltantes =  [ ]

sinonimo_nacionalidad = {
    # para agregar sinónimos la fórmula es 'si decía X' : 'poner Y'
    'alemania' : 'alemana','german':'alemana','germany':'alemana',
    'brazilian':'brasilera',
    'france' : 'francesa', 'french' : 'francesa', 'francia' : 'francesa','française':'francesa',
    'españa' : 'española',
    'mexicano' : 'mexicana',
    'polish':'polaca',
    'united kingdom':'britanica'
}

resumenes = generar_resumenes_basicos()

#for escuela in resumenes.keys() :
for escuela in escuelas :
    # Si la próxima línea falla con:
    #   NameError: name 'resumenes' is not defined
    # es xq falta ejecutar la celda 'Generación de Resúmenes Básicos de Inscripción'.
    resumen = resumenes[escuela]
    resumen['generos'] = { }
    generos = resumen['generos']
    nacionalidades = resumen['nacionalidades'] = { }
    for pasaporte in resumen['pasaportes'] :
        persona = locate_persona(pasaporte.strip(), requerimientos, incoming)
        # print('Persona:', persona)
        # genero = 'Otro'
        genero = 'Desconocido'
        if 'Sexo' in persona :
            genero = persona['Sexo']
        if not genero in generos :
            generos[genero] = 0
        generos[genero] += 1

        nacionalidad = 'desconocida'
        if 'Nacionalidad' in persona :
            nacionalidad = persona['Nacionalidad'].lower()
            if nacionalidad in sinonimo_nacionalidad :
                nacionalidad = sinonimo_nacionalidad[nacionalidad]
        if not nacionalidad in nacionalidades :
            nacionalidades[nacionalidad] = 0
        nacionalidades[nacionalidad] += 1


# Si la próxima línea falla con:
#   NameError: name 'resumen_UNSAM' is not defined
# es xq falta ejecutar la celda 'Generación de Resúmenes Básicos de Inscripción'.
# GENEROS_UNSAM = resumen_UNSAM['GENEROS'] = { }
# NACIONALIDADES_UNSAM = resumen_UNSAM['NACIONALIDADES'] = { }
# UNIVERSIDADES_UNSAM = resumen_UNSAM['UNIVERSIDADES'] = { }
GENEROS_UNSAM = resumenes['UNSAM']['GENEROS'] = { }
NACIONALIDADES_UNSAM = resumenes['UNSAM']['NACIONALIDADES'] = { }
UNIVERSIDADES_UNSAM = resumenes['UNSAM']['UNIVERSIDADES'] = { }
#PAISES UNIVERSIDADES_UNSAM = resumenes['UNSAM']['PAIS'] = { }

# resumen_UNSAM = {
#         'CANTIDAD_ORDENES' : 0,
#         'PASAPORTES' : set ()
#     }
for pasaporte in resumenes['UNSAM']['PASAPORTES'] :
    persona = locate_persona(pasaporte.strip(), requerimientos, incoming)
    # print('Persona:', persona)
    genero = 'Desconocido'
    if 'Sexo' in persona :
        genero = persona['Sexo']
    if not genero in GENEROS_UNSAM :
        GENEROS_UNSAM[genero] = 0
    GENEROS_UNSAM[genero] += 1

    nacionalidad = 'desconocida'
    if 'Nacionalidad' in persona :
        nacionalidad = persona['Nacionalidad'].lower()
    if nacionalidad in sinonimo_nacionalidad :
        nacionalidad = sinonimo_nacionalidad[nacionalidad]
    if not nacionalidad in NACIONALIDADES_UNSAM :
        NACIONALIDADES_UNSAM[nacionalidad] = 0
    NACIONALIDADES_UNSAM[nacionalidad] += 1

    universidad = 'Desconocida'
    if 'Universidad' in persona :
        universidad = persona['Universidad']
    if not universidad in UNIVERSIDADES_UNSAM :
        UNIVERSIDADES_UNSAM[universidad] = 0
    UNIVERSIDADES_UNSAM[universidad] += 1

    universidad = 'Desconocida'
    if 'Pais' in persona :
        universidad = persona['Pais']
    if not Pais in PAISES UNIVERSIDADES_UNSAM :
        PAISES UNIVERSIDADES_UNSAM[Pais] = 0
    PAISES UNIVERSIDADES_UNSAM[Pais] += 1


print("No se encontraron en 'incoming' los pasaportes:", pasaportes_faltantes)
print("Las personas con pasaporte faltante o datos en blanco se asociaron con 'Desconocido/a' género, nacionalidad y universidad.")

print('UNSAM')
print('    #inscripciones:', resumenes['UNSAM']['CANTIDAD_ORDENES'])
generos = resumenes['UNSAM']['GENEROS']
generos['Total'] = len(resumenes['UNSAM']['PASAPORTES'])
print('    estudiantes:', generos)
print('    nacionalidades:', resumenes['UNSAM']['NACIONALIDADES'])
#print('    #materias:', len(resumen['materias']))
print('    Universidades:')
universidades = resumenes['UNSAM']['UNIVERSIDADES']
for u in universidades.keys() :
#    pasaportes_inscriptos = materias[m]
    print('     -', u + ':', universidades[u])


#for escuela in resumenes.keys() :
for escuela in escuelas :
    print(escuela)
    resumen = resumenes[escuela]
    print('    #inscripciones:', resumen['cantidad_ordenes'])
    generos = resumen['generos']
    generos['Total'] = len(resumen['pasaportes'])
    print('    estudiantes:', generos)
    print('    nacionalidades:', resumen['nacionalidades'])
    print('    #materias:', len(resumen['materias']))
    print('    Inscriptos:')
    materias = resumen['materias']
    for m in materias.keys() :
        pasaportes_inscriptos = materias[m]
        contactos = set()
        for pasaporte in pasaportes_inscriptos :
            # req = next(row for row in requerimientos if row[index_pasaporte] == pasaporte)
            # nombre = req[index_nombre]
            # correo = req[index_correo]
            req = requerimientos[pasaporte]
            nombre = req['Nombre']
            correo = req['Correo']
            contacto = nombre + ' <' + correo + '>'
            #contacto = 'persona(s) sin información'
            #if pasaporte in incoming :
            #    persona = incoming[pasaporte]
            #    #contacto = (persona['Nombre'], persona['Apellido'], persona['Email'])
            #    contacto = persona['Nombre'] + ' ' + persona['Apellido'] + ' <' + persona['Email'] + '>'
            contactos.add(contacto)
        contactos_text = ', '.join(contactos)
        print('      -', m, ':', contactos_text)



SyntaxError: invalid syntax (<ipython-input-3-d9a12ed33d85>, line 105)

In [ ]:
#@title NO USAR -VIEJO-PASO 4: Generación de Resúmenes de Inscripción
##############################################

# Si la próxima línea falla con:
#   NameError: name 'incoming' is not defined
# es xq falta ejecutar la celda 'Lectura de incoming/listado.php'.
#print(incoming)

#incoming = leer_listado_incoming()
requerimientos = leer_requerimientos()
# print('incoming', incoming)
# Lista de personas no identificadas in incoming/listado.php
pasaportes_faltantes =  [ ]

sinonimo_nacionalidad = {
    # para agregar sinónimos la fórmula es 'si decía X' : 'poner Y'
    'alemania' : 'alemana','german':'alemana','germany':'alemana',
    'brazilian':'brasilera',
    'france' : 'francesa', 'french' : 'francesa', 'francia' : 'francesa','française':'francesa',
    'españa' : 'española',
    'mexicano' : 'mexicana',
    'polish':'polaca',
    'united kingdom':'britanica'

}


# Si la próxima línea falla con:
#   NameError: name 'resumenes' is not defined
# es xq falta ejecutar la celda 'Generación de Resúmenes Básicos de Inscripción'.
for escuela in resumenes.keys() :
    resumen = resumenes[escuela]
    resumen['generos'] = { }
    generos = resumen['generos']
    nacionalidades = resumen['nacionalidades'] = { }
    for pasaporte in resumen['pasaportes'] :
        persona = locate_persona(pasaporte.strip(), requerimientos, incoming)
        # print('Persona:', persona)
        # genero = 'Otro'
        genero = 'Desconocido'
        if 'Sexo' in persona :
            genero = persona['Sexo']
        if not genero in generos :
            generos[genero] = 0
        generos[genero] += 1

        nacionalidad = 'desconocida'
        if 'Nacionalidad' in persona :
            nacionalidad = persona['Nacionalidad'].lower()
            if nacionalidad in sinonimo_nacionalidad :
                nacionalidad = sinonimo_nacionalidad[nacionalidad]
        if not nacionalidad in nacionalidades :
            nacionalidades[nacionalidad] = 0
        nacionalidades[nacionalidad] += 1


# Si la próxima línea falla con:
#   NameError: name 'resumen_UNSAM' is not defined
# es xq falta ejecutar la celda 'Generación de Resúmenes Básicos de Inscripción'.
GENEROS_UNSAM = resumen_UNSAM['GENEROS'] = { }
NACIONALIDADES_UNSAM = resumen_UNSAM['NACIONALIDADES'] = { }
UNIVERSIDADES_UNSAM = resumen_UNSAM['UNIVERSIDADES'] = { }

# resumen_UNSAM = {
#         'CANTIDAD_ORDENES' : 0,
#         'PASAPORTES' : set ()
#     }
for pasaporte in resumen_UNSAM['PASAPORTES'] :
    persona = locate_persona(pasaporte.strip(), requerimientos, incoming)
    # print('Persona:', persona)
    genero = 'Desconocido'
    if 'Sexo' in persona :
        genero = persona['Sexo']
    if not genero in GENEROS_UNSAM :
        GENEROS_UNSAM[genero] = 0
    GENEROS_UNSAM[genero] += 1

    nacionalidad = 'desconocida'
    if 'Nacionalidad' in persona :
        nacionalidad = persona['Nacionalidad'].lower()
    if nacionalidad in sinonimo_nacionalidad :
        nacionalidad = sinonimo_nacionalidad[nacionalidad]
    if not nacionalidad in NACIONALIDADES_UNSAM :
        NACIONALIDADES_UNSAM[nacionalidad] = 0
    NACIONALIDADES_UNSAM[nacionalidad] += 1

    universidad = 'Desconocida'
    if 'Universidad' in persona :
        universidad = persona['Universidad']
    if not universidad in UNIVERSIDADES_UNSAM :
        UNIVERSIDADES_UNSAM[universidad] = 0
    UNIVERSIDADES_UNSAM[universidad] += 1


print("No se encontraron en 'incoming' los pasaportes:", pasaportes_faltantes)
print("Las personas con pasaporte faltante o datos en blanco se asociaron con 'Desconocido/a' género, nacionalidad y universidad.")

print('UNSAM')
print('    #inscripciones:', resumen_UNSAM['CANTIDAD_ORDENES'])
generos = resumen_UNSAM['GENEROS']
generos['Total'] = len(resumen_UNSAM['PASAPORTES'])
print('    estudiantes:', generos)
print('    nacionalidades:', resumen_UNSAM['NACIONALIDADES'])
#print('    #materias:', len(resumen['materias']))
print('    Universidades:')
universidades = resumen_UNSAM['UNIVERSIDADES']
for u in universidades.keys() :
#    pasaportes_inscriptos = materias[m]
    print('     -', u + ':', universidades[u])


for escuela in resumenes.keys() :
    print(escuela)
    resumen = resumenes[escuela]
    print('    #inscripciones:', resumen['cantidad_ordenes'])
    generos = resumen['generos']
    generos['Total'] = len(resumen['pasaportes'])
    print('    estudiantes:', generos)
    print('    nacionalidades:', resumen['nacionalidades'])
    print('    #materias:', len(resumen['materias']))
    print('    Inscriptos:')
    materias = resumen['materias']
    for m in materias.keys() :
        pasaportes_inscriptos = materias[m]
        contactos = set()
        for pasaporte in pasaportes_inscriptos :
            # req = next(row for row in requerimientos if row[index_pasaporte] == pasaporte)
            # nombre = req[index_nombre]
            # correo = req[index_correo]
            req = requerimientos[pasaporte]
            nombre = req['Nombre']
            correo = req['Correo']
            contacto = nombre + ' <' + correo + '>'
            #contacto = 'persona(s) sin información'
            #if pasaporte in incoming :
            #    persona = incoming[pasaporte]
            #    #contacto = (persona['Nombre'], persona['Apellido'], persona['Email'])
            #    contacto = persona['Nombre'] + ' ' + persona['Apellido'] + ' <' + persona['Email'] + '>'
            contactos.add(contacto)
        contactos_text = ', '.join(contactos)
        print('      -', m, ':', contactos_text)



[{'td': ['Fecha', 'Apellido', 'Nombre', 'Nacionalidad', 'Carrera', 'Universidad', 'Pais', 'Email', 'Email Alternativo', 'Periodo', 'Anio', 'Sexo', 'Pasaporte', 'Fecha Nacimiento', 'E-Mail Responsable']}, {'td': [None, 'López', 'Cecilia', 'Argentina', 'Psicopedagogía', 'Free Mover', 'Argentina', 'snielson@unsam.edu.ar', 'snielson@unsam.edu.ar', None, None, None, None, None, None]}, {'td': [None, 'Rodríguez Sánchez', 'Adrián Gerardo', 'Mexicana', 'Posgrado en Historiografía', 'Free Mover', 'Argentina', 'adrian.geros@gmail.com', 'adrian.geros@gmail.com', None, None, None, None, None, None]}, {'td': [None, 'Alban Montesinos', 'Camila Estefania', 'Ecuatoriana', 'Ingeniería Agroindustrial y de Alimentos', 'Universidad de Las Américas', 'Ecuador', 'camila.alban@udla.edu.ec', 'programasinternacionales@udla.edu.ec', '2do cuatrimestre (Agosto - Diciembre)', None, None, None, None, None]}, {'td': [None, 'Arteaga Pérez', 'Sebastián Adrián', 'Méxicana', 'Diseño de Artesanías', 'Universidad de Guada

In [ ]:
#@title VIEJO-NO USAR-Generación de Resúmenes Básicos de Inscripción
##########  ############################################

requerimientos = leer_google_spreadsheet(id_requerimientos_de_inscripcion)
encabezado = requerimientos[0]
#print(encabezado) # ['Correo', 'Nombre', 'Pasaporte', 'Escuela', 'Codigo', 'Materia', 'Horario', 'Prioridad']
index_escuela   = encabezado.index('Escuela') # es el nro de columna que en el encabezado se llama 'Escuela'
index_pasaporte = encabezado.index('Pasaporte')
index_materia   = encabezado.index('Materia')

# 'resumenes' es un Diccionario (Escuela -> Resumen); Resumen es, a su vez, otro diccionario.
# PENDIENTE: Sería mejor inicializar los resumenes de todas las escuelas acá xa que genere resumenes en blanco en caso de no tener incriptos en una escuela

"""
escuela: EH - Escuela de Humanidades
escuela: EPyG - Escuela de Política y Gobierno
escuela: EAyP - Escuela de Arte y Patrimonio
escuela: EIDAES - Escuela Interdisciplinaria de Altos Estudios Sociales
escuela: ECyT - Escuela de Ciencia y Tecnología
escuela: EEyN - Escuela de Economía y Negocios
"""

resumenes = { }
resumen_UNSAM = {
    # No es claro que sea útil pero las claves se definen en mayús para que sea
    # más fácil distinguirlas de las de los resúmenes de las escuelas.
        'CANTIDAD_ORDENES' : 0,
        'PASAPORTES' : set ()
    }

for row in requerimientos[1:] : # saltea el encabezado (requerimientos[0])
    escuela = row[index_escuela]
    if escuela == '' :
        print('row raro xq la escuela está vacía: ', row)
        break
    # Al ver x 1ra vez una escuela, inicializar su Resumen.
    if not escuela in resumenes :
        # Esta información sirve para cruzar datos con los datos de incoming.
        # Xej, con los pasaportes se obtienen las nacionalidades.
        resumenes[escuela] = {
            'cantidad_ordenes' : 0,
            'pasaportes' : set(), # es un conjunto
            'materias' : { } # dict: Materia -> Conjunto de pasaportes
        }
    resumen_escuela = resumenes[escuela]
    resumen_escuela['cantidad_ordenes'] = resumen_escuela['cantidad_ordenes'] + 1
    resumen_UNSAM['CANTIDAD_ORDENES'] = resumen_UNSAM['CANTIDAD_ORDENES'] + 1
    #resumen_escuela['cantidad_ordenes'] += 1
    #resumen_UNSAM['CANTIDAD_ORDENES'] += 1

    pasaporte = row[index_pasaporte]
    #print(pasaporte, escuela)
    resumen_escuela['pasaportes'].add(pasaporte)
    resumen_UNSAM['PASAPORTES'].add(pasaporte)
    materia = row[index_materia]
    materias_escuela = resumen_escuela['materias']
    if not materia in materias_escuela :
        materias_escuela[materia] = set ()
    materias_escuela[materia].add(pasaporte)

print('Resúmenes Básicos generados.')

Se recordó el contenido de la hoja Hoja 1 de 2024-C1-Requerimientos de Inscripción
Resúmenes Básicos generados.


In [ ]:
#@title LARA- Generación de mensajes de devolución de inscripciones por estudiante
############################################################################
# Para generar estos mensajes es necesario leer las órdenes de cada escuela.
# En el caso del SIU3 tenemos una situación actualizada.

def convertir_en_dict(rows : list, key_name : str, cols : list) -> dict :
    result = { }
    encabezado = rows[0]
    for row in rows[1:] : # saltea encabezado
        value = { }
        #print('row', row)
        for col in cols :
            value[col] = row[encabezado.index(col)]

        key = row[encabezado.index(key_name)]
        # OJO que key o value pueden ser banana en filas con celdas vacías.

        if not key in result :
            result[key] = [ ]
        result[key].append(value)
    return result

def leer_ordenes_SIU2(sheet_name : str) -> dict :
    ordenes_SIU2 = leer_google_spreadsheet(id_2023C1_ordenes_de_inscripcion, sheet_name)
#    atributos_eleccion = ['Escuela', 'Codigo', 'Materia', 'Horario', 'Prioridad'] # SIU3
    atributos_eleccion = ['Materia', 'Horario', 'Prioridad','Estado'] # SIU2
    result = convertir_en_dict(ordenes_SIU2, 'Pasaporte', atributos_eleccion)
    return result

escuelas_SIU2 = {
        'EIDAES' : 'Escuela Interdisciplinaria de Altos Estudios Sociales',
        'EPG' : 'Escuela de Política y Gobierno',   # es inconsistente que no se llame 'epYg'
        'ECYT' : 'Escuela de Ciencia y Tecnología',
        'EHYS' : 'Escuela de Hábitat y Sustentabilidad'
    }

ordenes = leer_ordenes_SIU3() # dict : pasaporte -> [ orden ]

for key in escuelas_SIU2.keys() :
    escuela = escuelas_SIU2[key]
    # print(escuela)
    ordenes_escuela = leer_ordenes_SIU2(key)
    # print(ordenes_escuela)
    for pasaporte in ordenes_escuela.keys() :
        if not pasaporte in ordenes :
            ordenes[pasaporte] = [ ]
        ordenes[pasaporte] += ordenes_escuela[pasaporte]
        # print('pasaporte', pasaporte, 'ordenes', ordenes[pasaporte])

# No es necesario leer los Requerimientos pero resulta cómodo para acceder a la
# información personal asociada a cada pasaporte.
requerimientos = leer_requerimientos()
# requerimientos[pasaporte] = { 'Correo' : str, 'Nombre' : str, 'Elecciones' : [ Eleccion ] }

for pasaporte in ordenes.keys() :
    elecciones = ordenes[pasaporte]
    principales = filter(lambda eleccion :
                            eleccion['Prioridad'] == 'Principal', elecciones)
    principales = list(map(lambda eleccion : eleccion['Materia'], principales))
    suplentes = list(filter(lambda eleccion :
                            eleccion['Prioridad'] == 'Suplente', elecciones))
    suplentes = list(map(lambda eleccion : eleccion['Materia'], suplentes))
    otras = list(filter(lambda eleccion :
                            not eleccion['Prioridad'] in ['Principal', 'Suplente'],
                            elecciones))
    for raro in otras :
        print('La materia', raro['Materia'], 'tiene prioridad', raro['Prioridad'],
              ', fuera de lo esperado: ["Principal", "Suplente"].')

    # materias = list(map(lambda eleccion : eleccion['Materia'], ordenes[pasaporte]))
    # print('Pasaporte:', pasaporte, ', Materias:', materias)
    if pasaporte in requerimientos :
        nombre = requerimientos[pasaporte]['Nombre']
        correo = requerimientos[pasaporte]['Correo']

        # print('Pasaporte:', pasaporte, ', principales:', principales, ', suplentes:', suplentes)
        # print('Correo:', correo, ', nombre:', nombre, ', principales:', principales, ', suplentes:', suplentes)
        print(correo, '\n', nombre, '\n', 'principales:', principales, '\n', 'suplentes:', suplentes, 'estados', estados)
    else :
        print('El pasaporte no se encuentra en los Requerimientos:', pasaporte)

    # valores = [1, 2, 3, 4, 5, 6, 7, 8, 9]
    # pares = list(filter(lambda x : x % 2 == 0, valores))
    # print(pares)    # [2, 4, 6, 8]

    # enteros = [1, 2, 4, 7]
    # cuadrados = list(map(lambda x : x ** 2, enteros))


# for pasaporte in requerimientos.keys() :
#     mensaje = ""
#     solicitud = requerimientos[pasaporte]
#     destinatario = '"' + solicitud['Nombre'] + '" <' + solicitud['Correo'] + '>'
#     mensaje += destinatario
#     siu2 = []
#     siu3 = []
#     for eleccion in solicitud['Elecciones'] :
#         # Eleccion.keys() == [ 'Escuela', 'Codigo', 'Materia', 'Horario', 'Prioridad' ]
#         nombre_materia = eleccion['Materia']
#         if eleccion['Escuela'] in escuelas_SIU2 :
#             siu2.append(nombre_materia)
#         else :
#             siu3.append(nombre_materia)
#             solicitud = ordenes[pasaporte]
#             for item in solicitud :
#                 print(item)
#             print('solicitud:', solicitud)
#             print('-----------------------------')

#     mensaje += '\n' + 'SIU2: ' + str(siu2) + '\n' + 'SIU3: ' + str(siu3) + '\n'
#     print(mensaje)
#     break

# print(ordenes)


Se recordó el contenido de la hoja SIU3 de 2023-1C-ORDENES DE INSCRIPCION
Se recordó el contenido de la hoja EIDAES de 2023-1C-ORDENES DE INSCRIPCION


ValueError: ignored

In [ ]:
#@title Traduce formulario de problemas con moodle a CSV
rows = leer_google_spreadsheet(id_problemas_moodle)
encabezado = ['Username', 'Password','Firstname','Lastname','Email', 'Course1', 'Type1']
out_rows = [encabezado] # es una lista de listas
out_rows_EAYP = [encabezado]
out_rows_EH = [encabezado]
out_rows_EEYN = [encabezado]
out_row_EIDAES = [encabezado]
out_row_EPYG = [encabezado]
out_rows_ECYT = [encabezado]
for row in rows[1:]: # saltea encabezado; cada fila es una lista de Strings
    #print('row: ', row)
    username = row[3]
    password = row [3]
    firstname = row [2]
    lastname = ''
    email = row [1]
    #for i in (5, 7, 10, 12, 14, 16):
    for i in (4, 6, 8, 11, 13, 15):
        escuela = row[i]
        j = i + 1 # inventamos la j porque en el formulario hay una columna más
        if i == 8 : j = i + 2
        course1 = row[j]
        if course1 == '' : continue
        aula_virtual = [username, password, firstname, lastname, email, course1]
        #print(','.join(aula_virtual))
        # if escuela =='EAyP - Escuela de Arte y Patrimonio' :
        #     out_rows_EAYP.append(aula_virtual)
        # else :
        #     out_rows.append(aula_virtual)

        if escuela =='EAyP - Escuela de Arte y Patrimonio' :
            out_rows_EAYP.append(aula_virtual)
        elif escuela == 'EH - Escuela de Humanidades' :
            out_rows_EH.append(aula_virtual)
        elif escuela == 'EEyN - Escuela de Economía y Negocios' :
            out_rows_EEYN.append(aula_virtual)
        elif escuela == 'EIDAES - Escuela Interdisciplinaria de Altos Estudios Sociales' :
            out_row_EIDAES.append(aula_virtual)
        elif escuela == 'EPyG - Escuela de Política y Gobierno' :
            out_row_EPYG.append(aula_virtual)
        else :
            out_rows_ECYT.append(aula_virtual)

print ('Arte y Patrimonio')
for row in out_rows_EAYP : print (row)
# for row in out_rows : print (row)

print ('humanidades')
for row in out_rows_EH: print (row)

print ('Economía y Negocios')
for row in out_rows_EEYN : print (row)

print ('EIDAES')
for row in out_row_EIDAES : print (row)

print ('Politica')
for row in out_row_EPYG : print (row)

print ('Ciencia y Tecnología')
for row in out_rows_ECYT: print (row)
#print ('todas')
for row in out_rows : print (row)

import csv

# Tus listas de resultados existentes...

# Definir una lista de tuplas que contenga el nombre del archivo y los resultados
listas_resultados = [
    ("resultados_EAYP.csv", out_rows_EAYP),
    ("resultados_EH.csv", out_rows_EH),
    ("resultados_EEYN.csv", out_rows_EEYN),
    ("resultados_EIDAES.csv", out_row_EIDAES),
    ("resultados_EPYG.csv", out_row_EPYG),
    ("resultados_ECYT.csv", out_rows_ECYT)
]

# Iterar a través de las listas de resultados y escribir cada una en un archivo CSV separado
for nombre_archivo, resultados in listas_resultados:
    with open(nombre_archivo, mode='w', newline='', encoding='utf-8') as archivo:
        escritor_csv = csv.writer(archivo)
        escritor_csv.writerows(resultados)
    print(f"Los resultados se han escrito en '{nombre_archivo}'")

from google.colab import files

# Nombres de los archivos que creaste
nombres_archivos = ["resultados_EAYP.csv", "resultados_EH.csv", "resultados_EEYN.csv", "resultados_EIDAES.csv", "resultados_EPYG.csv", "resultados_ECYT.csv"]

for nombre_archivo in nombres_archivos:
    files.download(nombre_archivo)


In [ ]:
# @title Esta celda es un intento por enviar correos masivos

# import sys
# print("Python version")
# print (sys.version) # Python version 3.10.12 (main, Jun 11 2023, 05:26:28) [GCC 11.4.0]

# probar https://hellomailman.com/

# from https://support.google.com/accounts/answer/185833
# An app password is a 16-digit passcode that gives a less secure app or device
# permission to access your Google Account. App passwords can only be used with
# accounts that have 2-Step Verification turned on.

# # from https://colab.research.google.com/github/j-agrawal/coding/blob/master/send_mail.ipynb
# import smtplib, ssl
# port = 465  # For SSL
# smtp_server = "smtp.gmail.com"
# #sender_email = input('enter sender mail address:')  # Enter your address
# sender_email = "incoming@unsam.edu.ar"
# # receiver_email = input('enter receiver mail address:')  # Enter receiver address
# receiver_email = "lweisstaub@unsam.edu.ar"
# password = input("Type your password and press enter: ")
# # message =input('enter myour message:')
# message = "hola"
# context = ssl.create_default_context()
# with smtplib.SMTP_SSL(smtp_server, port, context=context) as server:
#      #server.login(sender_email, password)
#      # SMTPAuthenticationError: (535, b'5.7.8 Username and Password not accepted. ' +
#      # Learn more at\n5.7.8  https://support.google.com/mail/?p=BadCredentials g2-20020a6b6b02000000b007791e286fdbsm2680238ioc.21 - gsmtp')
#      #server.sendmail(sender_email, receiver_email, message)

# amrith.gutierrez@uaem.edu.mx
#  Amrith Gutiérrez Arias
#  principales: ['Forma Urbana II ', 'Historia de la Arquitectura IV', ' Taller de Proyecto Arquitectónico VI', 'Materiales ']
#  suplentes: ['Aspectos éticos y legales del ejercicio profesional I', 'Instrumentos de Analís y Regulación Urbana ']
# johan.reynoso@uaem.edu.mx
#  Johan Daniel Reynoso David
#  principales: ['Forma Urbana II', 'Historia de la Arquitectura IV', 'Taller de Proyecto Arquitectónico VI ', 'Materiales']
#  suplentes: ['Aspectos éticos y legales del ejercicio profesional I', 'Instrumentos de análisis y regulación urbana']

In [ ]:
#@title Cruce de Incoming con Confirmados
#########################################

#https://docs.google.com/spreadsheets/d/1J1MJhgQe-y0IHnKHiq7CtanK1RVMAmuW-602QHilOnw
confirmados =  leer_google_spreadsheet(id_2023C1_incoming_confirmados)
incoming = leer_listado_incoming()

encabezado_confirmados = confirmados[0]
index_nombre_confirmado = encabezado_confirmados.index('Nombre')
index_apellido_confirmado = encabezado_confirmados.index('Apellido')
encabezado_incoming = leer_encabezado_incoming()
# print('encabezado_incoming', encabezado_incoming)
result = [ encabezado_incoming ]
for row in confirmados[1:] : # saltea encabezado
    nombre = row[index_nombre_confirmado].strip()       # elimina espacios en blanco antes y después
    apellido = row[index_apellido_confirmado].strip()   # elimina espacios en blanco antes y después
    # print(nombre, apellido)
    matches = [(pasaporte, persona) for (pasaporte, persona) in incoming.items()
            if persona['Nombre'].lower() == nombre.lower() and
                persona['Apellido'].lower() == apellido.lower()
        ]
    if len(matches) == 0 :
        matches = [(pasaporte, persona) for (pasaporte, persona) in incoming.items()
                if nombre.lower() in persona['Nombre'].lower() and
                    apellido.lower() in persona['Apellido'].lower()
            ]
    cruce_incoming = None
    if len(matches) == 0 :
        print('No se encontró en Incoming a la persona Confirmada:', nombre, apellido)
        cruce_incoming = ('', { 'Nombre' : nombre, 'Apellido' : apellido } )
    elif len(matches) == 1 :
         #print('matches[0]', matches[0])
         # matches[0] == ( pasaporte, incoming[pasaporte] )
         cruce_incoming = matches[0]
    else :
        print('Se encontraron múltiples resultados en Incoming para la persona Confirmada:', nombre, apellido)
        print('Resultados de la búsqueda:', matches)
        print('Se considera la información del 1er resultado.')
        cruce_incoming = matches[0]

    cruce_row = [ ]
    pasaporte = cruce_incoming[0]
    persona = cruce_incoming[1]
    for col in encabezado_incoming :
        value = ''
        if col == 'Pasaporte' :
            value = pasaporte
        elif col in persona :
            value = persona[col]
        cruce_row.append(value)
    # print(cruce_incoming)
    # print(cruce_row)
    result.append(cruce_row)

# ordenes = gc.open_by_key(requerimientos_de_inscripcion).sheet1
# ordenes.update(out_rows)
# print('Se generaron los Requerimientos de inscripción.')
import gspread
gc = gspread.authorize(creds)
sheet = gc.open_by_key(id_2023C1_incoming_confirmados).worksheet('Cruce incoming/listado.php')
sheet.update(result)
print('Se generó el cruce de información entre Confirmados e incoming/listado.php.')

Se leyó la hoja Hoja 1 de 2023-1C-Incoming CONFIRMADOS
Se recordó incoming/listado.php.
Se encontraron múltiples resultados en Incoming para la persona Confirmada: Lola RABER
Resultados de la búsqueda: [('18CE844305', {'Fecha': '2020-02-05 17:54:03', 'Apellido': 'RABER', 'Nombre': 'Lola', 'Nacionalidad': 'francesa', 'Carrera': 'Licenciatura de Ciencias Politicas', 'Universidad': 'Université Paris 8', 'Pais': 'Francia', 'Email': 'lola.raber@live.fr', 'Email Alternativo': 'lola.raber@etud.univ-paris8.fr', 'Periodo': '1er cuatrimestre (Marzo - Julio)', 'Anio': '2020', 'Sexo': 'Femenino', 'Fecha Nacimiento': '1999-02-20', 'E-Mail Responsable': 'ludovic.maillard@univ-Paris8.fr'}), ('18CE84430', {'Fecha': '2022-10-27 07:48:27', 'Apellido': 'RABER', 'Nombre': 'Lola', 'Nacionalidad': 'Francia', 'Carrera': 'Institute of Latin American Studies', 'Universidad': 'Université Sorbonne Nouvelle', 'Pais': 'Francia', 'Email': 'lola.raber@sorbonne-nouvelle.fr', 'Email Alternativo': 'lola.raber@live.fr',

In [ ]:
#@title Lectura de Requerimientos de Inscripción [VIEJO: no usar más]
#@markdown * Genera la variable `requerimientos` como una lista de filas.
#@markdown  La 1ra fila es el encabezado, el resto los requerimientos individuales.
#@markdown * Genera la variable `reqs_dict` (muy feo nombre) como un diccionario.
#@markdown  La clave es el pasaporte, el valor asociado es un diccionario con los
#@markdown  demás datos de solicitud: Correo, Nombre y Solicitudes (una lista de Requerimientos).

def leer_requerimientos() :
    # from https://stackoverflow.com/a/423596
    # You can use a global variable within functions by declaring it as `global`
    global requerimientos

    # from https://stackoverflow.com/a/843293
    # To check the existence of a local variable:
    # if 'myVar' in locals(): ...
    # To check the existence of a global variable:
    # if 'myVar' in globals(): ...
    if not 'requerimientos' in globals() :
        requerimientos = None
    if requerimientos != None :
        return requerimientos

    # from google.colab import auth
    # auth.authenticate_user()
    # import gspread
    # from google.auth import default
    # creds, _ = default()
    # gc = gspread.authorize(creds)
    #requerimientos_de_inscripcion = '1-gE4Nemi4qECqQBjFHQml4lEyF6k03QNBqqTS4Ojcuc'
    id_requerimientos_de_inscripcion = '1-gE4Nemi4qECqQBjFHQml4lEyF6k03QNBqqTS4Ojcuc'
    # sheet = gc.open_by_key(requerimientos_de_inscripcion).sheet1
    # requerimientos = sheet.get_all_values() # devuelve una lista de filas
    requerimientos = leer_google_spreadsheet(id_requerimientos_de_inscripcion)
    return requerimientos

# Al final, no es claro que el esfuerzo x usar vars globales haya servido de algo.
leer_requerimientos()

# La próxima idea es pasar los requerimientos a un lindo diccionario.
result = { }
#encabezado = requerimientos[0]
index = { }
for i, column in enumerate(requerimientos[0]) : # encabezado
    index[column] = i
#print(index) # {'Correo': 0, 'Nombre': 1, 'Pasaporte': 2, 'Escuela': 3, 'Codigo': 4, 'Materia': 5, 'Horario': 6, 'Prioridad': 7}

for row in requerimientos[1:] : # saltea encabezado
    pasaporte = row[index['Pasaporte']]
    if not pasaporte in result :
        result[pasaporte] = {
                'Correo' : row[index['Correo']],
                'Nombre' : row[index['Nombre']],
                'Solicitudes' : [ ]
            }
    requerimiento = {}
    atributos = ['Escuela', 'Codigo', 'Materia', 'Horario', 'Prioridad']
    for it in atributos :
        requerimiento[it] = row[index[it]]
    result[pasaporte]['Solicitudes'].append(requerimiento)

reqs_dict = result
key = list(result.keys())[0]
print('Ejemplo: reqs_dict[' + key + ']:', result[key])
print('requerimientos = ', requerimientos)

Ejemplo: reqs_dict[21EC69282]: {'Correo': 'Coline.Pigret-Cadou@etu.univ-paris1.fr', 'Nombre': 'Coline Pigret-Cadou', 'Solicitudes': [{'Escuela': 'Escuela de Humanidades', 'Codigo': 'MA-ET00210', 'Materia': 'Filosofía práctica/Etica', 'Horario': 'Martes noche', 'Prioridad': 'Principal'}, {'Escuela': 'Escuela de Humanidades', 'Codigo': 'MA-FC00254', 'Materia': 'Filosofía de la cultura', 'Horario': 'lunes noche', 'Prioridad': 'Principal'}, {'Escuela': 'Escuela de Política y Gobierno', 'Codigo': 'CP51', 'Materia': 'Política argentina (Armelino)', 'Horario': 'Martes mañana', 'Prioridad': 'Principal'}, {'Escuela': 'Escuela de Política y Gobierno', 'Codigo': '', 'Materia': 'Organismos internacionales (Rullansky)', 'Horario': 'miércoles noche', 'Prioridad': 'Principal'}, {'Escuela': 'Escuela de Política y Gobierno', 'Codigo': '', 'Materia': 'Comercio y finanzas internacionales', 'Horario': 'martes noche', 'Prioridad': 'Suplente'}, {'Escuela': 'Escuela de Política y Gobierno', 'Codigo': '', 'Ma

In [ ]:
#@title Generación de Resúmenes de Inscripciones por escuela [Vicen]
####################################################################

from google.colab import auth
auth.authenticate_user()
import gspread
from google.auth import default
creds, _ = default()
gc = gspread.authorize(creds)
#------------------
solicitudes_de_inscripcion = '1Uww4QkVCliRsD9lKracE6_yp5cwhap3F2hLLYl-_UIM'
#requerimientos_de_inscripcion = '1E7JSwmh2b-B-AF-LJm5EaIfnLt6KJPDU4ueB6uSQOt0'
#ordenes_de_inscripcion = '1uYK73OpeGNyeIhR2MU5hJeQw7C_WK0fOUMUhdyfB-p4'
sheet = gc.open_by_key(solicitudes_de_inscripcion).sheet1
rows = sheet.get_all_values() # devuelve una lista de filas
#print(rows)
#rows.pop(0) # saltea el encabezado de la tabla

alumnos = [0, 0, 0, 0, 0, 0, 0]
escuelas=['Escuela de Hábitat y Sustentabilidad','Escuela de Humanidades', 'Escuela de Arte y Patrimonio', 'Escuela de Ciencia y Tecnología', 'Escuela de Política y Gobierno','Escuela Interdisciplinaria de Altos Estudios Sociales','Escuela de Economía y Negocios']
for row in rows :
    for i in range(len(escuelas)):
        if escuelas[i] in row :
            alumnos[i] = alumnos[i]+ 1
for i in range(len(escuelas)):
    print(escuelas[i]+':', alumnos[i])
#######################################
vicen = {
    'Escuela de Hábitat y Sustentabilidad' : 0,
    'Escuela de Humanidades' : 0,
    'Escuela de Arte y Patrimonio' : 0,
    'Escuela de Ciencia y Tecnología' : 0,
    'Escuela de Política y Gobierno' : 0,
    'Escuela Interdisciplinaria de Altos Estudios Sociales' : 0,
    'Escuela de Economía y Negocios' : 0
}
for row in rows :
    for k in vicen.keys() :
        if k in row :
            vicen[k] = vicen[k] + 1
for k in vicen.keys() :
    print(k + ':', vicen[k])




Escuela de Hábitat y Sustentabilidad: 2
Escuela de Humanidades: 27
Escuela de Arte y Patrimonio: 16
Escuela de Ciencia y Tecnología: 10
Escuela de Política y Gobierno: 21
Escuela Interdisciplinaria de Altos Estudios Sociales: 24
Escuela de Economía y Negocios: 7
Escuela de Hábitat y Sustentabilidad: 2
Escuela de Humanidades: 27
Escuela de Arte y Patrimonio: 16
Escuela de Ciencia y Tecnología: 10
Escuela de Política y Gobierno: 21
Escuela Interdisciplinaria de Altos Estudios Sociales: 24
Escuela de Economía y Negocios: 7


In [ ]:
#@title Generación de órdenes de inscripción SIU2 y SIU3 [Javier] [Trabajo en curso]
###############################################
# Nueva versión de Javier
# 1- se van a generar de cero las órdenes de inscripción, a partir de los requerimientos.
# 2- se van a combinar las órdenes generadas con una eventual versión previa.
# 3- se va a definir un mecanismo de preservación, o no, de la versión previa, antes de sobreecribirse información.
###############################################

# from google.colab import auth
# auth.authenticate_user()
# import gspread
# from google.auth import default
creds, _ = default()
gc = gspread.authorize(creds)
#------------------
requerimientos_de_inscripcion = '1E7JSwmh2b-B-AF-LJm5EaIfnLt6KJPDU4ueB6uSQOt0'
#ordenes_de_inscripcion =
sheet = gc.open_by_key(requerimientos_de_inscripcion).sheet1
rows = sheet.get_all_values() # devuelve una lista de filas
#print(rows)
rows.pop(0) # saltea el encabezado de la tabla de Requerimientos
encabezado_SIU2 = ['Nombre', 'Pasaporte', 'Materia',  'Prioridad']
encabezado_SIU3 = ['Fecha_nacimiento','Universidad', 'Correo', 'Nombre', 'Pasaporte', 'Escuela', 'Codigo', 'Materia', 'Horario', 'Prioridad', 'Estado', 'Observaciones']
out_rows_EIDAES = [encabezado_SIU2]  # Escuela IDAES
out_rows_EPG = [encabezado_SIU2] # Escuela de Política y Gobierno
out_rows_ECYT = [encabezado_SIU2] # Escuela de Ciencia y Tecnología
out_rows_EHYS = [encabezado_SIU2] # Escuela de Hábitat y Sustentabilidad
out_rows_SIU3 = [encabezado_SIU3] # Ojo nueva variable

for row in rows:
    #Correo(0)	Nombre (1)	Pasaporte(2)	Escuela(3)	Codigo	Materia(5)	Horario	Prioridad(7)	Estado de la inscripción
    #out_humanidades.append(resultado) # este se vanombre = row[2]
    nombre = row[1]
    pasaporte = row[2]
    universidad = ''
    fecha_nacimiento = ''
    nacionalidad = ''
    if pasaporte in incoming :
        persona = incoming[pasaporte] # Acá están todos los datos personales de incoming.
        universidad = persona['Universidad']
        fecha_nacimiento = persona['Fecha Nacimiento']
        nacionalidad = persona['Nacionalidad']

    escuela = row[3]
    materia = row[5]
    prioridad = row[7]
    orden_SIU2 = [nombre, pasaporte, materia, prioridad]
    if escuela == 'Escuela Interdisciplinaria de Altos Estudios Sociales' :
        #simple.write(','.join(orden_SIU2) + "\n")
        out_rows_EIDAES.append(orden_SIU2)
    #OJO que ahora hay un else-if(elif) en vez de un nuevo if
    elif escuela == 'Escuela de Política y Gobierno' :
        out_rows_EPG.append(orden_SIU2)
    elif escuela == 'Escuela de Ciencia y Tecnología' :
        out_rows_ECYT.append(orden_SIU2)
    elif escuela == 'Escuela de Hábitat y Sustentabilidad' :
        out_rows_EHYS.append(orden_SIU2)
    else:
        # Formato SIU3: Correo,Nombre,Pasaporte,Escuela,Codigo,Materia,Horario,Prioridad,Estado de la inscripción
        correo = row[0]
        codigo = row[4]
        horario = row[6]
        orden_SIU3 = [fecha_nacimiento, universidad, correo, nombre, pasaporte, escuela, codigo, materia, horario, prioridad]
        out_rows_SIU3.append(orden_SIU3)



# **Pendiente**

* buscar otra forma de acceder a incoming/listado.php cuando falla el servidor de la UNSAM.
* combinar listados generados con listados modificados a mano.
* generar información de Matriculación, a partir de Incoming.
* envío automático de emails.
* workflows
* evitar nombres de materias creativos
* agregar un documento de word de los informes que genera para entregar a estudiantes
*  generar reportes para indicadores con inscripciones definitivas
* escupir info de indicadores en un excel.
*   en el paso 2 al generar el reporte cuando hay cambios de escuela reporta una fila arrba de lo que en verdad es. Por ejemplo dice que hubo cambios en la fila 17 en requerimientos y en realidad el cambio corresponde a la 18. El problema es el reporte, no el lugar donde es el cambio.
* agregar al reporte de cada escuela, la info de las unis de proveniencia para ver en el futuro qué unis tienen más match con qué escuelas.
*











# PASO 0: Instrucciones para iniciar una nueva campaña

**FASE 1: ARMADO DE LA INFRAESTRUCTURA**
1.   **Copiar** de la campaña anterior el formulario de google en el que lxs estudiantes completan su Solicitud de Inscripción. Dicho documento está guardado en la carpeta incoming del google drive de la oficina, dentro de la carpeta que corresponde al cuatrimestre de la campaña que se quiere copiar. Así por ejemplo, si se quiere copiar el formulario del primer cuatrimestre del 2024 se lo puede encontrar en Secretaria de Internacionalizacion/Movilidad/Incoming/2024-C2.

1.  **Mover** la copia del formulario a la carpeta de la nueva campaña. Así si se trata de la campaña del segundo cuatrimestre del 2024, la copia del formulario de  2024-C2-Solicitud de Inscripcion deberá guardarse en  Internacionalizacion/Movilidad/Incoming/2024-C2.

1.  **Cambiar el nombre** del archivo iniciando con el año de la campaña, luego seguido por la letra "C" que refiere a la palabra "cuatrimestre", luego agregar el número de cuatrimestre y finalmente la denominación del archivo, según el esquema que se detalla a continuación: año-CN°-Solicitud de Inscripción. Por ejemplo, si se trata de la campaña del primer cuatrimestre del año 2025, el archivo deberá llamarse: 2025-C1-Solicitud de Inscripcion.
1.   **Modificar el formulario**. Dentro del formulario de solicitud de inscripción, se debe realizar un cambio al iniciar la campaña. Se debe cambiar dentro de cada una de las 6 materias, la palabra "estado" por la palabra prioridad. Esto se debe a que al iniciar la campaña se les solicita a lxs estudiantes que seleccionen la prioridad de sus elecciones, y luego se utiliza el mismo casillero para solicitar confirmación o baja de las mateiras. Por lo tanto, cuando se copia el formulario, se debe modificar estos items.
2.   **Crear un spreadsheet vacio** donde se van a guardar los Requerimientos de inscripción. Nombrarla según el esquema: año-CN°-Requerimientos de Inscripción. Por ejemplo, si se trata de la campaña del primer cuatrimestre del año 2025, el archivo deberá llamarse: 2025-C1-Requerimientos de inscripcion
1.   **Crear otro spreadsheet vacio** donde se van a guardar las Órdenes de Inscripción, a partir de los Requerimientos. Nombrarla según el esquema: año-CN°-Ordenes de Inscripción. Por ejemplo, si se trata de la campaña del primer cuatrimestre del año 2025, el archivo deberá llamarse: 2025-C1-Ordenes de Inscripción
2. **Agregar solapas**. Dentro del documento año-N°C-Ordenes de Inscripción crear 3 solapas. Ponerle nombre a las solapas, siempre con mayusculas. Los nombres que lee el programa son los siguientes: SIU3, EPYG, EIDAES ¡No hay que ponerse creativo con los nombres!


**FASE 2: MODIFICACIONES EN EL CODIGO**
AGREGAR CAMBIOS una variable nueva que defina nombre documentos específicos del año.
 8. **Cambiar los nombres de las variables**. Dentro de la celda PASO 1: Inicialización hay que cambiar el año y/o el cuatrimestre de las variables que refieren a los documentos de la campaña anterior. Estas variables siguen el siguiente esquema:

*   id_solicitudes_de_inscripcion_AÑOCX
*   id_requerimientos_de_inscripcion_AÑOCX
*   id_ordenes_de_inscripcion_AÑOCX

Así por ejemplo, si estamos armando la campaña del segundo cuatrimestre del 2024, la variable que en el código figura como id_solicitudes_de_inscripcion_2024C1 debe pasar a llamarse,id_solicitudes_de_inscripcion_2024C2.
Debe aplicarse el mismo procedimiento para lista de variables mencionadas más arriba.  

9. **Cambiar las direcciones de lectura de las variables**.  Una vez que se cambió el nombre de la variable debe también modificarse la id de lectura para que el programa entienda dónde tiene que leer o escribir la información. Para esto hay que abrir en el navegador el documento cuya dirección se quiere extraer, copiar el id del mismo y reemplazarlo en la variable que corresponda dentro celda de inicialización. Siempre con comillas simples.
*Por ejemplo*, si queremos saber cuál es el id del spreadsheet 2024-C1-Solicitudes de inscripción tenemos que abrir el documento en el navegador. Allí veremos la siguiente información: https://docs.google.com/spreadsheets/d/1NsyECaE3SEu9VrU7whhQmlh1bmHkww7vuBc2M9wuStQ/edit#gid=784754961. El id que necesitamos es la cadena de números y letras intermedia  '1NsyECaE3SEu9VrU7whhQmlh1bmHkww7vuBc2M9wuStQ'
La variable debe quedar escrita de la siguiente manera:
id_solicitudes_de_inscripcion_2024C1 ='1NsyECaE3SEu9VrU7whhQmlh1bmHkww7vuBc2M9wuStQ'

10. Repetir el procedimiento para las tres variables mencionadas en el punto 8:
*  id_solicitudes_de_inscripcion_AÑOCX
*  id_requerimientos_de_inscripcion_AÑOCX
*  id_ordenes_de_inscripcion_AÑOCX


1. VER qué hacer con INCOMING
  Cambiar en la sección cruce de incomings con confirmados la lectura del google spreadsheet id_2024_C2_incomng confirmados x id_2025_C1_incoming_confirmados
2.   cambiar id de solicitudes_de_inscripcion y de requerimientos_de_inscripcion en la celda que genera requerimientos
1.   cambiar id de requerimientos_de_inscripcion en celda de generacion de ordenes inscripcion siu2 y siu3Elemento de la lista

2.   ACTUALIZAR EN ESTA SECCIÓN LOS NOMBRES DE LAS VARIABLES PARA QUE SE ENTIENDA EL PRÓXIMO CUATRIMESTRE O POR ALGUIEN QUE NUNCA LO HIZO





